In [ ]:
%env UNSLOTH_DISABLE_FAST_GENERATION=1

env: UNSLOTH_DISABLE_FAST_GENERATION=1


In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
#unsloth-2025.8.1 unsloth_zoo-2025.8.1
!pip install --upgrade --force-reinstall --no-deps --no-cache-dir unsloth unsloth_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.6/183.6 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: unsloth_zoo
    Found existing installation: unsloth_zoo 2025.8.6
    Uninstalling unsloth_zoo-2025.8.6:
      Successfully uninstalled unsloth_zoo-2025.8.6
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.8.7
    Uninstalling unsloth-2025.8.7:
      Successfully uninstalled unsloth-2025.8.7


In [ ]:
!pip install -q datasets==3.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth 2025.8.7 requires tyro, which is not installed.
unsloth-zoo 2025.8.6 requires msgspec, which is not installed.
unsloth-zoo 2025.8.6 requires tyro, which is not installed.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 r

In [ ]:
from unsloth import FastLanguageModel, FastModel
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os
import torch
from torch import tensor
import torch.nn.functional as F
from transformers import TrainingArguments, Trainer, ModernBertModel, AutoModelForSequenceClassification, training_args
from datasets import load_dataset, Dataset
from tqdm import tqdm

model_name = 'answerdotai/ModernBERT-base'

NUM_CLASSES = 3
DATA_DIR = "data/"

model, tokenizer = FastModel.from_pretrained(
    model_name = model_name,load_in_4bit = False,
    max_seq_length = 2048,
    dtype = None,
    auto_model = AutoModelForSequenceClassification,
    num_labels = NUM_CLASSES,
    full_finetuning=True
)
print("model parameters:" + str(sum(p.numel() for p in model.parameters())))

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.7: Fast Modernbert patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Float16 full finetuning uses more memory since we upcast weights to float32.


model.safetensors:   0%|          | 0.00/599M [00:00<?, ?B/s]

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model parameters:149607171


In [38]:
ds = load_dataset("neoyipeng/financial_reasoning_aggregated")

In [39]:
ds = ds.filter(lambda x: x["source"] == 6)

Filter:   0%|          | 0/27957 [00:00<?, ? examples/s]

Filter:   0%|          | 0/6990 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8737 [00:00<?, ? examples/s]

# Experiment 1: Train on all data except fpb

In [ ]:
from datasets import load_dataset

ds = load_dataset("neoyipeng/financial_reasoning_aggregated")

label_dict = {'NEUTRAL/MIXED':1,
              'NEGATIVE':0,
              'POSITIVE':2}

ds = ds.filter(lambda x: x["task"] == "sentiment")
ds = ds.filter(lambda x: x["source"] != 6) # skip fpb
ds = ds.map(lambda ex: {
    "text":   ex["text"],
    "labels":  np.eye(NUM_CLASSES)[label_dict[ex["label"]]],
},remove_columns=["label","prompt","aspect","summary_detail","title","topic","score_topic", "source", "task","__index_level_0__"])


# Inspect the first example
print(ds["train"][0])

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/4.09M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/5.16M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/27957 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/6234 [00:00<?, ? examples/s]

Map:   0%|          | 0/19326 [00:00<?, ? examples/s]

Map:   0%|          | 0/4834 [00:00<?, ? examples/s]

Map:   0%|          | 0/6041 [00:00<?, ? examples/s]

{'text': 'Thieves made off with jewelry and artifacts worth a reported $1.1 billion after an early-morning burglary at a Germ… https://t.co/a5j7a9evKs', 'labels': [0.0, 1.0, 0.0]}


In [ ]:
sentiment_ds_trn = ds["train"]#.shuffle(seed=42).select(range(4_500))
sentiment_ds_val = ds["validation"]#.shuffle(seed=42).select(range(500))

In [ ]:
def tokenize_function(examples): return tokenizer(examples['text'])

In [ ]:
sentiment_ds_trn = sentiment_ds_trn.map(tokenize_function, batched=True)
sentiment_ds_val = sentiment_ds_val.map(tokenize_function, batched=True)

Map:   0%|          | 0/19326 [00:00<?, ? examples/s]

Map:   0%|          | 0/4834 [00:00<?, ? examples/s]

In [ ]:
import os
os.environ['WANDB_API_KEY'] = 'c6003ee33aa4304f3b55296330fbf7595523dcfc'

import wandb
wandb.login()

wandb.init(
    project="modernfinbert",
    name="train without fpb",
)

wandb: Currently logged in as: neoyipeng to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=sentiment_ds_trn,
    eval_dataset=sentiment_ds_val,
    args=TrainingArguments(
        per_device_train_batch_size=32,
        gradient_accumulation_steps=1,
        warmup_steps=10,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim=training_args.OptimizerNames.ADAMW_TORCH,
        learning_rate=5e-5,
        weight_decay=0.001,
        lr_scheduler_type="cosine",
        seed=3407,
        num_train_epochs=10,
        save_strategy="epoch",
        report_to="wandb",
        group_by_length=True,
        eval_strategy="steps",
        eval_steps=0.25,
        logging_strategy="steps",
        logging_steps=0.25,
    ),
    compute_metrics=lambda eval_pred: { "accuracy": accuracy_score(eval_pred[1].argmax(axis=-1), eval_pred[0].argmax(axis=-1)) }
)

/tmp/ipython-input-3744878363.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 19,326 | Num Epochs = 10 | Total steps = 6,040
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 1 x 1) = 32
 "-____-"     Trainable parameters = 149,607,171 of 149,607,171 (100.00% trained)


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Accuracy
1510,0.133300,0.954414,0.712867
3020,0.034500,1.352152,0.714315
4530,0.004400,1.711948,0.720728
6040,0.000300,1.775909,0.722590


In [ ]:
model = model.cuda()
model = model.eval()
FastLanguageModel.for_inference(model)
print()

# run on fpb

In [ ]:
from transformers.trainer_utils import get_last_checkpoint

output_dir = "trainer_output"
last_checkpoint = get_last_checkpoint(output_dir)
print("Last checkpoint:", last_checkpoint)

model, tokenizer = FastModel.from_pretrained(
    model_name = last_checkpoint,load_in_4bit = False,
    max_seq_length = 2048,
    dtype = None,
    auto_model = AutoModelForSequenceClassification,
    num_labels = NUM_CLASSES,
)

Last checkpoint: trainer_output/checkpoint-6040
==((====))==  Unsloth 2025.8.7: Fast Modernbert patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


In [ ]:
from datasets import concatenate_datasets
from datasets import load_dataset

fpb  = load_dataset("financial_phrasebank", "sentences_50agree",trust_remote_code=True)['train']

In [ ]:
len(fpb)

4846

In [ ]:
# Evaluate on FinancialPhraseBank
model = model.cuda().eval()
FastLanguageModel.for_inference(model)

batch_size = 32
correct = 0
results = []

fpb_texts = fpb['sentence']
fpb_labels = fpb['label']

with torch.no_grad():
    for i in tqdm(range(0, len(fpb_texts), batch_size), desc="Evaluating FPB"):
        batch_texts = fpb_texts[i:i+batch_size]
        batch_labels = fpb_labels[i:i+batch_size]

        # Tokenize
        inputs = tokenizer(batch_texts, return_tensors="pt", padding=True,
                          truncation=True, max_length=2048)
        inputs = {k: v.cuda() for k, v in inputs.items()}

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=-1).cpu().numpy()

        # Count correct
        correct += np.sum(preds == batch_labels)

        # Store results
        for j in range(len(batch_texts)):
            results.append({
                "text": batch_texts[j][:200],
                "true": batch_labels[j],
                "pred": preds[j],
                "probs": probs[j].detach().float().cpu().numpy(),
                "correct": preds[j] == batch_labels[j]
            })

accuracy = 100 * correct / len(fpb_texts)
print(f"\nFinancialPhraseBank accuracy: {accuracy:.2f}% ({correct}/{len(fpb_texts)})")

Evaluating FPB: 100%|██████████| 152/152 [00:09<00:00, 16.67it/s]


FinancialPhraseBank accuracy: 90.47% (4384/4846)


In [ ]:
# Show detailed results
from sklearn.metrics import classification_report, confusion_matrix
import random

y_true = [r['true'] for r in results]
y_pred = [r['pred'] for r in results]

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['NEGATIVE', 'NEUTRAL', 'POSITIVE']))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# Show some random examples
print("\n--- Random sample predictions ---")
for sample in random.sample(results, min(10, len(results))):
    status = "✅" if sample['correct'] else "❌"
    print(f"\nText: {sample['text']}...")
    print(f"True: {sample['true']} | Pred: {sample['pred']} {status}")
    print(f"Confidence: {sample['probs'][sample['pred']]:.3f}")


Classification Report:
              precision    recall  f1-score   support

    NEGATIVE       0.88      0.90      0.89       604
     NEUTRAL       0.92      0.93      0.92      2879
    POSITIVE       0.88      0.86      0.87      1363

    accuracy                           0.90      4846
   macro avg       0.89      0.90      0.89      4846
weighted avg       0.90      0.90      0.90      4846


Confusion Matrix:
[[ 542   56    6]
 [  58 2666  155]
 [  19  168 1176]]

--- Random sample predictions ---

Text: Net sales decreased to EUR 220.5 mn from EUR 470.0 mn in the corresponding period in 2009 ....
True: 0 | Pred: 0 ✅
Confidence: 1.000

Text: We are honored to be recognized by Gartner as a specialist in Business Process Modeling and Analysis , in the company of many of the largest software vendors in the world ....
True: 2 | Pred: 2 ✅
Confidence: 1.000

Text: Strong growth has continued also in China ....
True: 2 | Pred: 2 ✅
Confidence: 1.000

Text: With Aldata , we can help 

# save model in huggingface hub

In [ ]:
# Login to HuggingFace Hub
from huggingface_hub import login, HfApi
import os

# You'll need to set your HF token
# os.environ['HUGGINGFACE_HUB_TOKEN'] = 'your_token_here'  # Uncomment and add your token

login()  # Uncomment this line when ready to upload

print("Ready to upload model to HuggingFace Hub")

Ready to upload model to HuggingFace Hub


In [ ]:
# Create model card and upload model
model_name = "neoyipeng/ModernFinBERT-base"  # Change to your username

model_card_content = f"""
---
language: en
license: apache-2.0
tags:
- finance
- sentiment-analysis
- modernbert
- financial-nlp
datasets:
- neoyipeng/financial_reasoning_aggregated
metrics:
- accuracy
widget:
- text: "The company reported strong quarterly earnings with revenue beating expectations."
  example_title: "Positive Example"
- text: "Stock prices fell sharply following disappointing guidance from management."
  example_title: "Negative Example"
- text: "The merger is expected to close in Q3 pending regulatory approval."
  example_title: "Neutral Example"
---

# ModernFinBERT: Financial Sentiment Analysis

ModernFinBERT is a financial sentiment analysis model based on the ModernBERT architecture, fine-tuned specifically for financial text classification.

## Model Details

- **Base Model**: answerdotai/ModernBERT-base
- **Task**: 3-class sentiment classification (Negative, Neutral, Positive)
- **Training Data**: Financial text from multiple sources (excluding FinancialPhraseBank for evaluation)
- **Parameters**: {sum(p.numel() for p in model.parameters()):,}

## Performance

- **FinancialPhraseBank Accuracy**: {accuracy:.2f}%
- **Target**: >94% accuracy on FinancialPhraseBank

## Usage

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("{model_name}")
model = AutoModelForSequenceClassification.from_pretrained("{model_name}")

text = "The company's quarterly results exceeded analyst expectations."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

labels = ["NEGATIVE", "NEUTRAL", "POSITIVE"]
predicted_class = labels[predictions.argmax().item()]
confidence = predictions.max().item()

print(f"Sentiment: {{predicted_class}} ({{confidence:.2f}})")
```

## Training Details

- **Epochs**: 10
- **Batch Size**: 32
- **Learning Rate**: 5e-5
- **Optimizer**: AdamW
- **Scheduler**: Cosine
- **Framework**: Unsloth + Transformers

## Citation

If you use this model, please cite:

```bibtex
@misc{{modernfinbert2025,
  title={{ModernFinBERT: A Modern Approach to Financial Sentiment Analysis}},
  author={{Neo Yi Peng}},
  year={{2025}},
  howpublished={{HuggingFace Model Hub}},
  url={{https://huggingface.co/{model_name}}}
}}
```
"""

# Save model card
with open("README.md", "w") as f:
    f.write(model_card_content)

print("Model card created successfully!")

Model card created successfully!


In [ ]:
# Upload model to HuggingFace Hub
# Uncomment the lines below when ready to upload

# Push model and tokenizer to hub
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

# Upload model card
api = HfApi()
api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",
    repo_id=model_name,
    repo_type="model",
)

print(f"Model uploaded successfully to: https://huggingface.co/{model_name}")

README.md: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmpn5m12row/model.safetensors    :  11%|#1        | 33.5MB /  299MB            

Saved model to https://huggingface.co/neoyipeng/ModernFinBERT-base


README.md: 0.00B [00:00, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


Model uploaded successfully to: https://huggingface.co/neoyipeng/ModernFinBERT-base
